<a href="https://colab.research.google.com/github/jovian-explorer/Sun/blob/main/MOM/%20Solar_wind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
file to plot using the 'output_expdopp_4point.csv'
exp_dopp =  downconfreq + dopp_firstmoment - tfreq 
where treq = 8410932002.0000 #tfreq = transmitted freq given in .obs file of Akatsuki X Band 8.4 GHz


structure of the csv-  
i,timesec,power_total,downconfreq,dopp_firstmoment,exp_dopp, second_moment,RTEC
'''

"\nfile to plot using the 'output_expdopp_4point.csv'\nexp_dopp =  downconfreq + dopp_firstmoment - tfreq \nwhere treq = 8410932002.0000 #tfreq = transmitted freq given in .obs file of Akatsuki X Band 8.4 GHz\n\n\nstructure of the csv-  \ni,timesec,power_total,downconfreq,dopp_firstmoment,exp_dopp, second_moment,RTEC\n"

In [2]:
#-----------------------------------------------------------------------------------------------------------------
#importing libraries
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
#-----------------------------------------------------------------------------------------------------------------

In [3]:
#importing data from my GitHub
url = 'https://raw.githubusercontent.com/jovian-explorer/Sun/main/MOM/output_expdopp_4point.csv'
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_colwidth', None)
dataset = pd.read_csv(url,delimiter = ',')
print(dataset.head(1))
#-----------------------------------------------------------------------------------------------------------------

   1   21720.0  43.40549452131652  8410640000.0  -42400.492706323195  \
0  1  21720.25           41.14198  8.410640e+09        -42396.381766   

   -334402.4927062988  4.726165950751449  2.751054161769941e+30  
0      -334398.381765           5.229972           2.993352e+30  


In [4]:
#-----------------------------------------------------------------------------------------------------------------
# accessing the different columns of the csv dataset I made above
dataset.columns = ["i","timesec","power_total","downconfreq","dopp_firstmoment","exp_dopp","second_moment","RTEC"]
#storing values for each column into a separate arrays

'''
Finding Moments of powSpec (Power spectral density distribution)- by defining "powSpecmoment" function - 
zeroth moment = power, 
first moment = doppler freq
second = width of freq spread 
downconfreq - RF_TO_IF downconv frequency as read from header file
final experimental/Observed doppler shift observed in received signal is (given by heterodyning principle)  
exp_dopp = Downconversion frequency + dopp_firstmoment_freq - transmitted freq
IV - write output in "output.txt" file containing :  time in seconds----power(first moment)-----measured dopp freq
'''
i = np.array(dataset.i)           		  	              # Values of iteration
timesec= np.array(dataset.timesec)               	      # Values of time in sec 6300.1, 6300.2
power_total= np.array(dataset.power_total)                # Values of zeroth moment 
downconfreq= np.array(dataset.downconfreq)                # Values of downconfreq
dopp_firstmoment= np.array(dataset.dopp_firstmoment)      # Values of first moment 
exp_dopp= np.array(dataset.exp_dopp)                	  # Values of exp_dopp
second_moment = np.array(dataset.second_moment)	          # Values of second_moment
RTEC = np.array(dataset.RTEC)                             # Values of Radial Total electron content
#-----------------------------------------------------------------------------------------------------------------

In [5]:
#------------------------------------------------
m = 9.1*10**(-31)
e = 1.6*10**(-19)
frec = 8410640000.0
k= 5.9
l_2= 2.4235* 10**11
alpha = 1.14*10**(-24)
L = 3.9195*10**11
ds=(1.469* 10**11)/len(i)
R = 1.252* 10**10
#------------------------------------------------

In [17]:
density = RTEC/R
plasma_frequency_model_1 = np.sqrt(4*np.pi*density*(e**2)/m)#Yamato et al. 2002
plasma_frequency_model_2 = np.sqrt(density*(e**2)/(np.pi*m))#Chen F. F., 1974

#refractive index equation taken from Hollweg et al. 1970
refractive_index_model_1 = np.sqrt(1 - (plasma_frequency_model_1/frec)**2)
refractive_index_model_2 = np.sqrt(1 - (plasma_frequency_model_2/frec)**2)

refractive_index_model_1 = np.nan_to_num(refractive_index_model_1)
refractive_index_model_2 = np.nan_to_num(refractive_index_model_2)

d_refractive_index_model_1 = np.gradient(refractive_index_model_1,1)
d_refractive_index_model_2 = np.gradient(refractive_index_model_2,1)
dy = np.gradient(second_moment)

#angular broadening taken from Hollweg et al. 1970

temp_1 = d_refractive_index_model_1/dy
temp_2 = d_refractive_index_model_2/dy
temp_1 = temp_1*ds
temp_2 = temp_2*ds

angular_broadening_model_1 = np.sum(temp_1)
angular_broadening_model_2 = np.sum(temp_2)

#Solar wind velocity taken from Bird et al. 1982
solar_wind_velocity_model_1 = (5.77*frec*L)/(angular_broadening_model_1*k*l_2) 
solar_wind_velocity_model_2 = (5.77*frec*L)/(angular_broadening_model_2*k*l_2)

In [7]:
# STATUS OF CODE -


# RUNS PROPERLY
# DEFINING THE VARIABLES PROPERLY PENDING
# CHECKING THE UNITS PENDING AS WELL

In [18]:
print(temp_1)
print(temp_2)

print(angular_broadening_model_1)
print(angular_broadening_model_2)

print(solar_wind_velocity_model_1)
print(solar_wind_velocity_model_2)

[-1.16121074 -1.14670712 -1.15806389 ... -1.0772098  -1.07726352
 -1.07441843]
[-0.0294138  -0.02904639 -0.02933409 ... -0.02728602 -0.02728738
 -0.02721528]
1582258457.9730306
1582274137.1785364
8.407426216908599
8.407342905325395
